## set up emial alter for price change
- pulls up-to-date stock data using pandas_datareader (backed by yfinance).
- ends an email alert once when the condition is met.
- uses environment variables for credentials (good for security).

In [6]:
#install packacge needed
#conda activate base   ##activate environment if needed
#conda install -c conda-forge yfinance -y
#conda install -c conda-forge pandas_datareader -y(not successful, installed in terminal)


CondaError: Run 'conda init' before 'conda activate'


Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import smtplib
from email.message import EmailMessage
import yfinance as yf
import datetime as dt
from pandas_datareader import data as pdr
import time

In [5]:
# Email credentials from environment variables
EMAIL_ADDRESS = os.environ.get('EMAIL_USER')
EMAIL_PASSWORD = os.environ.get('EMAIL_PASS')
#EMAIL_PASSWORD = 'your-app-password-here' if the emial need Two-Factor Authentication like gmail

In [11]:
# Stock and target price settings
##testing## stock = "QQQ" , target_price = 180
stocks = {
    "QQQ": 180,
    "SPY": 400,
    "AAPL": 150,
    "GOOGL": 2500
}

In [12]:
# Email setup
msg = EmailMessage()
msg['Subject'] = f'Alert on {stock}!'
msg['From'] = EMAIL_ADDRESS
msg['To'] = 'recipient@example.com'  # Set the recipient email address

# Time settings
start = dt.datetime(2018, 12, 1)
now = dt.datetime.now()

In [13]:
# Initialize alerted flags
alerted_flags = {stock: False for stock in stocks}

# Main loop
while True:
    for stock, target_price in stocks.items():
        try:
            ticker = yf.Ticker(stock)
            df = ticker.history(period="1mo")  # Get historical data for the last month

            if df.empty:
                print(f"{dt.datetime.now()}: No data available for {stock}")
                continue  # Skip this stock if there's no data available

            current_price = df["Close"][-1]  # Get the most recent closing price

            # Check if the price has reached the target and if the alert hasn't been triggered
            if current_price > target_price and not alerted_flags[stock]:
                alerted_flags[stock] = True
                message = f"{stock} has reached the alert price of {target_price}\nCurrent Price: {current_price}"
                print(message)
                msg.set_content(message)

                # Send email alert
                try:
                    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
                        smtp.login(EMAIL_ADDRESS, EMAIL_PASSWORD)  # Using app password if 2FA is enabled
                        smtp.send_message(msg)
                        print(f"Email sent for {stock} successfully.")
                except smtplib.SMTPAuthenticationError as auth_error:
                    print(f"Authentication failed: {auth_error}")
                except Exception as e:
                    print(f"Error occurred while sending email for {stock}: {e}")
            else:
                print(f"{dt.datetime.now()}: No new alerts for {stock}. Current Price: {current_price}")

        except Exception as e:
            print(f"Error occurred for {stock}: {e}")

    # Wait for 60 seconds before checking again
    time.sleep(60)

/tmp/ipykernel_4277/149506504.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  current_price = df["Close"][-1]  # Get the most recent closing price


QQQ has reached the alert price of 180
Current Price: 488.8299865722656
Error occurred while sending email for QQQ: (334, b'UGFzc3dvcmQ6')


/tmp/ipykernel_4277/149506504.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  current_price = df["Close"][-1]  # Get the most recent closing price


SPY has reached the alert price of 400
Current Price: 566.760009765625
Error occurred while sending email for SPY: (334, b'UGFzc3dvcmQ6')


/tmp/ipykernel_4277/149506504.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  current_price = df["Close"][-1]  # Get the most recent closing price
/tmp/ipykernel_4277/149506504.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  current_price = df["Close"][-1]  # Get the most recent closing price


...

/tmp/ipykernel_4277/149506504.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  current_price = df["Close"][-1]  # Get the most recent closing price
/tmp/ipykernel_4277/149506504.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  current_price = df["Close"][-1]  # Get the most recent closing price
/tmp/ipykernel_4277/149506504.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  current_price = df["Close"][-1]  # Get the most r

2025-05-03 21:24:13.967839: No new alerts for QQQ. Current Price: 488.8299865722656
2025-05-03 21:24:14.052884: No new alerts for SPY. Current Price: 566.760009765625
2025-05-03 21:24:14.152825: No new alerts for AAPL. Current Price: 205.35000610351562


/tmp/ipykernel_4277/149506504.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  current_price = df["Close"][-1]  # Get the most recent closing price


2025-05-03 21:24:14.253205: No new alerts for GOOGL. Current Price: 164.02999877929688


KeyboardInterrupt: 

In [10]:
##testing for one stock##
# Initialize alerted flag
alerted = False

# Main loop
while True:
    try:
        ticker = yf.Ticker(stock)
        current_price = ticker.history(period="1mo")["Close"][-1]

        if current_price > target_price and not alerted:
            alerted = True
            message = f"{stock} has reached the alert price of {target_price}\nCurrent Price: {current_price}"
            print(message)
            msg.set_content(message)

            with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
                smtp.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
                smtp.send_message(msg)
                print("Email sent successfully.")
        else:
            print(f"{dt.datetime.now()}: No new alerts. Current Price: {current_price}")
    except Exception as e:
        print(f"Error occurred: {e}")

    time.sleep(60)

/tmp/ipykernel_4277/472561347.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  current_price = ticker.history(period="1mo")["Close"][-1]


QQQ has reached the alert price of 180
Current Price: 488.8299865722656
Error occurred: (334, b'UGFzc3dvcmQ6')


/tmp/ipykernel_4277/472561347.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  current_price = ticker.history(period="1mo")["Close"][-1]


2025-05-03 21:12:53.388343: No new alerts. Current Price: 488.8299865722656


/tmp/ipykernel_4277/472561347.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  current_price = ticker.history(period="1mo")["Close"][-1]


...

/tmp/ipykernel_4277/472561347.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  current_price = ticker.history(period="1mo")["Close"][-1]


2025-05-03 21:21:54.154273: No new alerts. Current Price: 488.8299865722656


/tmp/ipykernel_4277/472561347.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  current_price = ticker.history(period="1mo")["Close"][-1]


2025-05-03 21:22:54.229619: No new alerts. Current Price: 488.8299865722656


KeyboardInterrupt: 